<center><h1>Лабораторна робота №1</h1>
<h2>Наука про дані: підготовчий етап</h2></center>

<center><h3>Роботу виконав студент групи ФБ-24 Синеуцький Олександр</h3></center>

<h4>1. Створити env, в якому будуть встановлені всі необхідні бібліотеки та налаштування для данної лабораторної роботи.</h4>

> conda create --name lab2 python<br>
> conda activate lab2<br>
> conda deactivate

<h4>2. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура
має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження.</h4>

In [38]:
import urllib.request
import os
from datetime import datetime


URL = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean"
DateAndTime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


if not os.path.exists("DataCSV"):
    os.makedirs("DataCSV")


for ProvinceID in range(1, 28):
    url = URL.format(ProvinceID)
    FileName = f"NOAA_ID{ProvinceID}_{DateAndTime}.csv"
    FilePath = os.path.join("DataCSV", FileName)

    try:
        urllib.request.urlretrieve(url, FilePath)
    except Exception as e:
        print(f"Помилка при скачуванні файлу для ProvinceID {ProvinceID}: {str(e)}")


print("Файли успішно скачано.")

Файли успішно скачано.


<h4>3. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли.</h4>

In [42]:
import pandas as pd
import os


def clean(DirPath):
    ls = []

    if not os.path.exists(DirPath) or not os.path.isdir(DirPath):
        print(f"Директорії {DirPath} не існує або не є директорією.")
    else:
        files = os.listdir(DirPath)

        for i, file in enumerate(files):

            if file.endswith(".csv"):
                FilePath = os.path.join(DirPath, file)
                df = pd.read_csv(FilePath, index_col=False, header=1)
                df["ID"] = i + 1
                ls.append(df)

        df = pd.concat(ls).drop_duplicates().reset_index(drop=True)

    # print(list(df.columns.values))
    df = df.rename(columns={" VHI<br>": "VHI"})
    df = df.rename(columns={" SMN": "SMN"})

    # print(df.head())
    df["year"].replace({"<tt><pre>1982": "1982"}, inplace=True)
    df = df.drop(df.loc[df["VHI"] == -1].index)

    # print(df.tail())
    df = df.drop(60398)

    # print(df.dtypes)
    df["year"] = pd.to_numeric(df["year"], errors="coerce")

    # print(f'{df.head()}\n{df.dtypes}\n{list(df.columns.values)}\n{df.tail()}')
    selected_records = df[(df["year"] == 2000) & (df["week"] == 18)]
    print("\nЗапис за умовою: \n", selected_records)
    # print(df)


clean("DataCSV")


Запис за умовою: 
          year  week    SMT     SMT    VCI    TCI    VHI  ID
953    2000.0  18.0  0.382  291.46  87.96  40.29  64.13   1
3190   2000.0  18.0  0.321  291.75  62.07  44.97  53.52   2
5427   2000.0  18.0  0.258  291.37  57.87  35.04  46.45   3
7664   2000.0  18.0  0.274  294.94  58.05  43.47  50.76   4
9901   2000.0  18.0  0.230  293.42  48.85  43.10  45.98   5
12138  2000.0  18.0  0.422  289.89  89.19  46.43  67.81   6
14375  2000.0  18.0  0.292  296.09  58.30  49.03  53.67   7
16612  2000.0  18.0  0.306  296.63  58.66  40.70  49.68   8
18849  2000.0  18.0  0.280  292.93  56.17  39.62  47.89   9
21086  2000.0  18.0  0.372  290.84  82.90  38.86  60.88  10
23323  2000.0  18.0  0.309  292.81  58.99  45.14  52.06  11
25560  2000.0  18.0  0.376  290.30  80.00  60.17  70.09  12
27797  2000.0  18.0  0.341  291.00  77.63  41.08  59.36  13
30034  2000.0  18.0  0.368  292.21  81.01  35.14  58.07  14
32271  2000.0  18.0  0.399  288.67  70.34  39.18  54.76  15
34508  2000.0  18.0 

<h4>4. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні:</h4>

In [43]:
import pandas as pd
import os


def clean(DirPath):
    ls = []

    if not os.path.exists(DirPath) or not os.path.isdir(DirPath):
        print(f"Директорії {DirPath} не існує або не є директорією.")
    else:
        files = os.listdir(DirPath)

        for i, file in enumerate(files):

            if file.endswith(".csv"):
                FilePath = os.path.join(DirPath, file)
                df = pd.read_csv(FilePath, index_col=False, header=1)
                df["ID"] = i + 1
                ls.append(df)

        df = pd.concat(ls).drop_duplicates().reset_index(drop=True)

    # print(list(df.columns.values))
    df = df.rename(columns={" VHI<br>": "VHI"})
    df = df.rename(columns={" SMN": "SMN"})

    # print(df.head())
    df["year"].replace({"<tt><pre>1982": "1982"}, inplace=True)
    df = df.drop(df.loc[df["VHI"] == -1].index)

    # print(df.tail())
    df = df.drop(60398)

    # print(df.dtypes)
    df["year"] = pd.to_numeric(df["year"], errors="coerce")

    # print(f'{df.head()}\n{df.dtypes}\n{list(df.columns.values)}\n{df.tail()}')
    # selected_records = df[(df["year"] == 2000) & (df["week"] == 18)]
    # print("\nЗапис за умовою: \n", selected_records)
    # print(df)
    return df


def index_replace(df):
    new_index = {
        1: 22,
        2: 24,
        3: 23,
        4: 25,
        5: 3,
        6: 4,
        7: 8,
        8: 19,
        9: 20,
        10: 21,
        11: 9,
        12: 9,
        13: 10,
        14: 11,
        15: 12,
        16: 13,
        17: 14,
        18: 15,
        19: 16,
        20: 25,
        21: 17,
        22: 18,
        23: 6,
        24: 1,
        25: 2,
        26: 7,
        27: 5,
    }
    df["ID"].replace(new_index, inplace=True)
    print(df)


index_replace(clean("DataCSV"))

         year  week    SMT     SMT    VCI    TCI    VHI  ID
0      1982.0   1.0  0.059  258.24  51.11  48.78  49.95  22
1      1982.0   2.0  0.063  261.53  55.89  38.20  47.04  22
2      1982.0   3.0  0.063  263.45  57.30  32.69  44.99  22
3      1982.0   4.0  0.061  265.10  53.96  28.62  41.29  22
4      1982.0   5.0  0.058  266.42  46.87  28.57  37.72  22
...       ...   ...    ...     ...    ...    ...    ...  ..
60348  2024.0   3.0  0.094  273.85  54.19   9.03  31.61   5
60349  2024.0   4.0  0.094  274.23  55.17   9.50  32.34   5
60350  2024.0   5.0  0.097  275.12  56.11   8.67  32.39   5
60351  2024.0   6.0  0.103  276.40  56.88   8.94  32.91   5
60352  2024.0   7.0  0.109  276.77  54.76  14.47  34.62   5

[57833 rows x 8 columns]


<h4>5. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):<br>
o Ряд VHI для області за рік, пошук екстремумів (min та max);<br>
o Ряд VHI за всі роки для області, виявити роки з екстремальними посухами, які торкнулися більше вказаного відсотка області;<br>
o Аналогічно для помірних посух</h4>

In [6]:
import pandas as pd  # імпортуємо бібліотеку pandas. ми її використаємо 3 рази, 1 для того, щоб прочитати csv файл, 2 щоб обєднати всі датафрейми в один великий датафрейм, 3 щоб перевести значення стопчика year до чисел
import os   # імпортуємо бібліотеку os. ми її використовуємо 4 рази, 1 перевіряємо чи існує директорія, 2 - чи це директорія, 3 дивимся вміст директорії, 4 об'єднуємо директорію та файл, щоб зробити єдиний шлях


def clean(DirPath): # функція яка зробить data cleaning
    ls = [] # список для всіх датафреймів

    if not os.path.exists(DirPath) or not os.path.isdir(DirPath): # перевіряємо чи існує директорія та чи є директорією
        print(f"Директорії {DirPath} не існує або не є директорією.")
    else:
        files = os.listdir(DirPath) # запишемо вміст директорії в список files

        for i, file in enumerate(files): # пройдемося циклом по всім файлам, i індекс, file - файл

            if file.endswith(".csv"): # перевіряємо чи файл закінчується на .csv
                FilePath = os.path.join(DirPath, file) # об'єднуємо шлях до директорії з файлом в єдиний шлях
                df = pd.read_csv(FilePath, index_col=False, header=1) # читаємо наш файл, index_col=False означає що ми будемо ігнорувати стовпець індексів у файлі, header=1 означає що перший рядок файлу це наш заголовок 
                df["ID"] = i + 1 # додамо стовпчик з ID області
                ls.append(df) #додамо кожен фрейм у список, для подальшого об'єднання

        df = pd.concat(ls).drop_duplicates().reset_index(drop=True) # об'єднаємо всі врейми зі списку в один фрейм, дропнемо дублікати, те дропнемо старі індекси, drop=True означає що попередні індекси будуть видалені

    # print(list(df.columns.values)) # виведемо список назв стовпців
    df = df.rename(columns={" VHI<br>": "VHI"}) # змінимо назву колонки   старе:нове
    df = df.rename(columns={" SMN": "SMN"})

    # print(df.head())
    df["year"].replace({"<tt><pre>1982": "1982"}, inplace=True) # замінемо в стовпчику рік, значення старе:нове, inplace=True означає що зміна відбудеться в нашому фреймі в не створиться новий
    df = df.drop(df.loc[df["VHI"] == -1].index) # дропаєм індекси в яких містяться значення VHI -1

    # print(df.tail())
    df = df.drop(60398) # дропаєм конкретний індекс

    # print(df.dtypes)
    df["year"] = pd.to_numeric(df["year"], errors="coerce") # переводимо значення в стовпчику рік до числових, errors=coerce означає що якщо конвертація неможлива то значення будуть NaN но е намбер

    # print(f'{df.head()}\n{df.dtypes}\n{list(df.columns.values)}\n{df.tail()}') # тут я вивожу все знову для перевірки чи все гаразд
    # selected_records = df[(df["year"] == 2000) & (df["week"] == 18)] # роблю вибірку зазначену в методичці
    # print("\nЗапис за умовою: \n", selected_records)
    # print(df)
    return df


def index_replace(df):
    new_index = {
        1: 22,
        2: 24,
        3: 23,
        4: 25,
        5: 3,
        6: 4,
        7: 8,
        8: 19,
        9: 20,
        10: 21,
        11: 9,
        12: 9,
        13: 10,
        14: 11,
        15: 12,
        16: 13,
        17: 14,
        18: 15,
        19: 16,
        20: 25,
        21: 17,
        22: 18,
        23: 6,
        24: 1,
        25: 2,
        26: 7,
        27: 5,
    }
    df["ID"].replace(new_index, inplace=True)
    # print(df)
    return df


def VhiOblPerYear(df, year, province_id):
    min_v = df[(df.year == year) & (df["ID"] == province_id)]["VHI"].min() # спочатку ми вибираєм датафрейм в якому рік = нашому та ID дорівнює нашому, потім ми вибираємо тільки стовпчик VHI, а потім вибираємо тількі мінімальне значення з нього
    max_v = df[(df.year == year) & (df["ID"] == province_id)]["VHI"].max() # спочатку ми вибираєм датафрейм в якому рік = нашому та ID дорівнює нашому, потім ми вибираємо тільки стовпчик VHI, а потім вибираємо тількі максимальне значення з нього

    print(f"Мінімальне значення: {min_v}")
    print(f"Максимальне значення: {max_v}")


def ExtrSafari(df, provinceID, percents):
    middleValue = df[(df["ID"] == provinceID)].groupby("year")["VHI"].mean() # спочатку ми вибираєм датафрейм в якому ID = нашому, отримали конкретну область за всі роки, потім ми згопували їх по рокам, типу там же 12 тижнів, то ми згрупували їх по рокам, потім взали їх VHI, потім взяли їх середнє значення, і утворився часковий датафрейм де тільки роки і їх середнє VHI
    years = middleValue[middleValue > percents] # це наш частковий датафрейм, де ми вибрали тількі те що задовольнає умові
    print(years.index.tolist()) # тут ми з нашого часткового датафрейму взяли тільки індекси, та засунули їх в список
    # print(middleValue)


df = index_replace(clean("DataCSV"))
VhiOblPerYear(df, 2020, 10)
ExtrSafari(df, 5, 55)

Мінімальне значення: 24.52
Максимальне значення: 72.11
[1997.0, 2004.0]
